### Load das bibliotecas

In [1]:
import pandas as pd
import pickle

In [2]:
pd.set_option('display.max_columns', 150) # ajusta a exibicao para um numero de colunas superior a quantidade padrao

### Load dos datasets
#### SAGE

In [3]:
dados_sage = pd.read_csv('../../data/FINAL/lista_dados_sage_MENSAL.csv', sep=';')
dados_sage.uf = dados_sage.uf.astype(str)

# exibe a dimensao e uma amostra
print('dimensao (linhas, colunas):', dados_sage.shape)
dados_sage.head()

dimensao (linhas, colunas): (17728, 6)


,ibge,uf,cidade,n_ano,n_mes,total_confirmado_abs
0,110001,RO,Alta Floresta D'Oeste,2016,10,0
1,110001,RO,Alta Floresta D'Oeste,2016,11,0
2,110001,RO,Alta Floresta D'Oeste,2016,12,0
3,110002,RO,Ariquemes,2016,10,0
4,110002,RO,Ariquemes,2016,11,0


#### IBGE

In [4]:
dados_ibge = pickle.load(open("../../data/FINAL/IBGE_Mun", "rb"))
dados_ibge.drop(columns=['ANO', 'UF', 'Codmun7', 'Município'], inplace=True)
dados_ibge.rename(columns = {'T_AGUA': 't_agua',
                             'T_BANAGUA': 't_banagua',
                             'T_DENS': 't_dens',
                             'T_LIXO': 't_lixo',
                             'T_LUZ': 't_luz',
                             'AGUA_ESGOTO': 'agua_esgoto'}, inplace=True)

# exibe a dimensao e uma amostra
print('dimensao (linhas, colunas):', dados_ibge.shape)
dados_ibge.head()

dimensao (linhas, colunas): (5565, 7)


,Codmun6,t_agua,t_banagua,t_dens,t_lixo,t_luz,agua_esgoto
11130,110001.0,93.69,80.20,22.58,94.05,93.98,9.31
11131,110002.0,98.54,85.35,27.15,96.72,98.58,11.40
11132,110003.0,95.49,91.63,19.93,99.14,96.36,3.31
11133,110004.0,97.96,93.44,20.50,98.17,98.89,1.85
11134,110005.0,97.53,86.87,16.66,91.89,98.85,5.34


#### Pluviometria (chuvas)

In [5]:
dados_pluviometria = pd.read_csv('../../data/FINAL/dfChuvasFinal.csv', sep=';')
dados_pluviometria.rename(columns = {'Evaporacao Piche': 'evaporacao_piche',
                                     'Insolacao': 'insolacao',
                                     'Precipitacao': 'precipitacao',
                                     'Temp Comp Media': 'temp_comp_media',
                                     'TempMaxima': 'temp_maxima',
                                     'TempMinima': 'temp_minima',
                                     'Umidade Relativa Media': 'umidade_relativa_media',
                                     'Velocidade do Vento Media': 'velocidade_do_vento_media'}, inplace=True)

# exibe a dimensao e uma amostra
print('dimensao (linhas, colunas):', dados_pluviometria.shape)
dados_pluviometria.head()

dimensao (linhas, colunas): (311, 11)


,evaporacao_piche,insolacao,precipitacao,temp_comp_media,temp_maxima,temp_minima,umidade_relativa_media,velocidade_do_vento_media,Ano,Mes,UF
0,0.000000,2.680899,7.360674,24.926292,31.068539,23.105618,87.132022,0.373788,2016,1,AC
1,4.666452,6.088387,5.526452,26.336129,31.812258,22.159355,76.472581,2.296774,2016,1,AL
2,0.000000,2.876590,5.387277,25.839084,30.837405,21.756997,76.968830,0.908026,2016,1,AM
3,3.303226,4.851613,3.751613,26.696774,31.987097,23.883871,78.637097,1.962366,2016,1,AP
4,0.087532,4.308052,11.418182,18.510571,22.702208,21.115455,55.770455,0.892133,2016,1,BA


#### Google Trends

In [6]:
dados_trends = pd.read_csv('../../data/FINAL/region_trend.csv', sep=';')
aux = dados_trends['month_year'].str.split("-", expand = True)
dados_trends['month'] = aux[0].astype(int)
dados_trends['year'] = aux[1].astype(int)
dados_trends.drop(columns=['location', 'keyword', 'geo', 'gprop', 'month_year', 'cod_uf'], inplace=True)
dados_trends.rename(columns = {"hits": "google_trends_score"}, inplace = True)

# exibe a dimensao e uma amostra
print('dimensao (linhas, colunas):', dados_trends.shape)
dados_trends.head()

dimensao (linhas, colunas): (324, 4)


,google_trends_score,sigla_uf,month,year
0,71,DF,3,2016
1,48,DF,1,2016
2,39,DF,4,2016
3,66,DF,5,2016
4,58,DF,8,2016


### Bases auxiliares
#### Base para preencher as lacunas de meses inexistentes na base original (necessário para gerar os dados absolutos dos 12 meses)

In [7]:
# lista com os anos distintos da base original
n_ano_list = dados_sage.n_ano.unique().tolist()

# lista fixa com os numeros dos meses de um ano (de 1 a 12)
n_mes_list = list(range(1, 13))

# lista com os codigos ibge distintos da base original
ibge_list = dados_sage.ibge.unique().tolist()

# gera uma lista aninhada em 3 niveis, combinando cada elemento das 3 listas anteriores para formar um dicionario
# o objetivo aqui e gerar uma linha para cada combinacao distinta de {n_ano, n_mes, ibge}
list_of_list = [[[{'n_ano': i, 'n_mes': j, 'ibge': k} for i in n_ano_list] for j in n_mes_list] for k in ibge_list]

# transforma a lista aninhada em uma lista simples de dicionarios
list_of_dict = [item for l3 in list_of_list for l2 in l3 for item in l2]

# gera o dataframe a partir da lista de dicionarios
df_aux = pd.DataFrame(list_of_dict)

# lista com os conjuntos distintos de ['ibge', 'uf', 'cidade'] da base original
lista_cidades = dados_sage.groupby(['ibge', 'uf', 'cidade']).first().reset_index().drop(columns = ['n_ano', 'n_mes', 'total_confirmado_abs'])

# junta a uf e o nome da cidade ao codigo ibge dos meses gerados
df_ibge_ano_mes = pd.merge(df_aux, lista_cidades, on = ['ibge'], how = 'inner')

# exibe a dimensao e uma amostra
print('dimensao (linhas, colunas):', df_ibge_ano_mes.shape)
df_ibge_ano_mes

dimensao (linhas, colunas): (27276, 5)


,ibge,n_ano,n_mes,uf,cidade
0,110001,2016,1,RO,Alta Floresta D'Oeste
1,110001,2016,2,RO,Alta Floresta D'Oeste
2,110001,2016,3,RO,Alta Floresta D'Oeste
3,110001,2016,4,RO,Alta Floresta D'Oeste
4,110001,2016,5,RO,Alta Floresta D'Oeste
5,110001,2016,6,RO,Alta Floresta D'Oeste
6,110001,2016,7,RO,Alta Floresta D'Oeste
7,110001,2016,8,RO,Alta Floresta D'Oeste
8,110001,2016,9,RO,Alta Floresta D'Oeste
9,110001,2016,10,RO,Alta Floresta D'Oeste


### Join dos datasets

In [8]:
# junta (merge) os dataframes df_ibge_ano_mes e dados_sage, gerando linhas com NaN para os meses em que
# a base final nao possui registros
dados_final = pd.merge(df_ibge_ano_mes, dados_sage, on = ['ibge', 'n_ano', 'n_mes', 'uf', 'cidade'], how = 'left')

# ordena a base para garantir que os valores dos meses anteriores (lagging) serao correspondentes aos meses adjacentes
dados_final.sort_values(by = ['ibge', 'n_ano', 'n_mes'], inplace = True)

# exibe o resultado parcial
print('dimensao (linhas, colunas):')
print('df_ibge_ano_mes:', df_ibge_ano_mes.shape)
print('dados_sage:', dados_sage.shape)
print('dados_final:', dados_final.shape)
dados_final.head(24)

dimensao (linhas, colunas):
df_ibge_ano_mes: (27276, 5)
dados_sage: (17728, 6)
dados_final: (27276, 6)


,ibge,n_ano,n_mes,uf,cidade,total_confirmado_abs
0,110001,2016,1,RO,Alta Floresta D'Oeste,NaN
1,110001,2016,2,RO,Alta Floresta D'Oeste,NaN
2,110001,2016,3,RO,Alta Floresta D'Oeste,NaN
3,110001,2016,4,RO,Alta Floresta D'Oeste,NaN
4,110001,2016,5,RO,Alta Floresta D'Oeste,NaN
5,110001,2016,6,RO,Alta Floresta D'Oeste,NaN
6,110001,2016,7,RO,Alta Floresta D'Oeste,NaN
7,110001,2016,8,RO,Alta Floresta D'Oeste,NaN
8,110001,2016,9,RO,Alta Floresta D'Oeste,NaN
9,110001,2016,10,RO,Alta Floresta D'Oeste,0.0


In [9]:
# junta a base parcial com os dados de ibge
dados_final2 = pd.merge(dados_final, dados_ibge, left_on = 'ibge', right_on = 'Codmun6', how = 'left')

# descarta as colunas utilizadas no merge
dados_final2.drop(columns = ['Codmun6'], inplace = True)

# exibe o resultado parcial
print('dimensao (linhas, colunas):')
print('dados_final:', dados_final.shape)
print('dados_ibge:', dados_ibge.shape)
print('dados_final2:', dados_final2.shape)
dados_final2.head(24)

dimensao (linhas, colunas):
dados_final: (27276, 6)
dados_ibge: (5565, 7)
dados_final2: (27276, 12)


,ibge,n_ano,n_mes,uf,cidade,total_confirmado_abs,t_agua,t_banagua,t_dens,t_lixo,t_luz,agua_esgoto
0,110001,2016,1,RO,Alta Floresta D'Oeste,NaN,93.69,80.20,22.58,94.05,93.98,9.31
1,110001,2016,2,RO,Alta Floresta D'Oeste,NaN,93.69,80.20,22.58,94.05,93.98,9.31
2,110001,2016,3,RO,Alta Floresta D'Oeste,NaN,93.69,80.20,22.58,94.05,93.98,9.31
3,110001,2016,4,RO,Alta Floresta D'Oeste,NaN,93.69,80.20,22.58,94.05,93.98,9.31
4,110001,2016,5,RO,Alta Floresta D'Oeste,NaN,93.69,80.20,22.58,94.05,93.98,9.31
5,110001,2016,6,RO,Alta Floresta D'Oeste,NaN,93.69,80.20,22.58,94.05,93.98,9.31
6,110001,2016,7,RO,Alta Floresta D'Oeste,NaN,93.69,80.20,22.58,94.05,93.98,9.31
7,110001,2016,8,RO,Alta Floresta D'Oeste,NaN,93.69,80.20,22.58,94.05,93.98,9.31
8,110001,2016,9,RO,Alta Floresta D'Oeste,NaN,93.69,80.20,22.58,94.05,93.98,9.31
9,110001,2016,10,RO,Alta Floresta D'Oeste,0.0,93.69,80.20,22.58,94.05,93.98,9.31


In [10]:
# junta a base parcial com os dados de pluviometria
dados_final3 = pd.merge(dados_final2, dados_pluviometria, left_on = ['uf', 'n_mes', 'n_ano'], right_on = ['UF', 'Mes', 'Ano'], how = 'left')

# descarta as colunas utilizadas no merge
dados_final3.drop(columns=['UF', 'Mes', 'Ano'], inplace = True)

# exibe o resultado parcial
print('dimensao (linhas, colunas):')
print('dados_final2:', dados_final2.shape)
print('dados_pluviometria:', dados_pluviometria.shape)
print('dados_final3:', dados_final3.shape)
# Filtramos os dados do estado "AC" para facilitar a validação,
# uma vez que o estado "RO" nao possui dados de pluviometria
dados_final3.loc[dados_final3.uf.str.contains('AC')].head(24)

dimensao (linhas, colunas):
dados_final2: (27276, 12)
dados_pluviometria: (311, 11)
dados_final3: (27276, 20)


,ibge,n_ano,n_mes,uf,cidade,total_confirmado_abs,t_agua,t_banagua,t_dens,t_lixo,t_luz,agua_esgoto,evaporacao_piche,insolacao,precipitacao,temp_comp_media,temp_maxima,temp_minima,umidade_relativa_media,velocidade_do_vento_media
144,120001,2016,1,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,2.680899,7.360674,24.926292,31.068539,23.105618,87.132022,0.373788
145,120001,2016,2,AC,Acrelândia,0.0,93.87,55.29,41.61,97.44,94.56,23.82,0.0,2.206780,8.993220,26.018305,31.108475,23.845763,86.546610,0.585648
146,120001,2016,3,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,1.914516,8.191935,25.499677,30.335484,23.674194,83.064516,0.976330
147,120001,2016,4,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,2.676667,7.538333,26.633333,32.528333,22.968333,85.350000,0.800240
148,120001,2016,5,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,2.909677,3.083871,25.130968,30.801613,21.746774,85.141129,0.838039
149,120001,2016,6,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,3.400000,1.300000,24.508000,31.046667,19.793333,82.395833,0.937424
150,120001,2016,7,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,4.153226,0.659677,25.308387,33.604839,19.533871,75.451613,0.741236
151,120001,2016,8,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,3.541270,1.885714,25.599683,33.752381,20.039683,71.103175,0.909116
152,120001,2016,9,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,2.971667,2.125000,26.052667,33.465000,21.125000,80.437500,0.840252
153,120001,2016,10,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,2.835484,8.988710,26.809677,33.322581,22.991935,83.290323,0.732939


In [11]:
# junta a base parcial com os dados do google trends
dados_final4 = pd.merge(dados_final3, dados_trends, left_on = ['uf', 'n_mes', 'n_ano'], right_on = ['sigla_uf', 'month', 'year'], how = 'left')

# descarta as colunas utilizadas no merge
dados_final4.drop(columns=['sigla_uf', 'month', 'year'], inplace = True)

# exibe o resultado parcial
print('dimensao (linhas, colunas):')
print('dados_final3:', dados_final3.shape)
print('dados_trends:', dados_trends.shape)
print('dados_final4:', dados_final4.shape)
# Filtramos os dados do estado "AC" para facilitar a validação,
# uma vez que o estado "RO" nao possui dados de pluviometria
dados_final4.loc[dados_final4.uf.str.contains('AC')].head(24)

dimensao (linhas, colunas):
dados_final3: (27276, 20)
dados_trends: (324, 4)
dados_final4: (27276, 21)


,ibge,n_ano,n_mes,uf,cidade,total_confirmado_abs,t_agua,t_banagua,t_dens,t_lixo,t_luz,agua_esgoto,evaporacao_piche,insolacao,precipitacao,temp_comp_media,temp_maxima,temp_minima,umidade_relativa_media,velocidade_do_vento_media,google_trends_score
144,120001,2016,1,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,2.680899,7.360674,24.926292,31.068539,23.105618,87.132022,0.373788,44
145,120001,2016,2,AC,Acrelândia,0.0,93.87,55.29,41.61,97.44,94.56,23.82,0.0,2.206780,8.993220,26.018305,31.108475,23.845763,86.546610,0.585648,53
146,120001,2016,3,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,1.914516,8.191935,25.499677,30.335484,23.674194,83.064516,0.976330,93
147,120001,2016,4,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,2.676667,7.538333,26.633333,32.528333,22.968333,85.350000,0.800240,44
148,120001,2016,5,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,2.909677,3.083871,25.130968,30.801613,21.746774,85.141129,0.838039,38
149,120001,2016,6,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,3.400000,1.300000,24.508000,31.046667,19.793333,82.395833,0.937424,100
150,120001,2016,7,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,4.153226,0.659677,25.308387,33.604839,19.533871,75.451613,0.741236,100
151,120001,2016,8,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,3.541270,1.885714,25.599683,33.752381,20.039683,71.103175,0.909116,0
152,120001,2016,9,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,2.971667,2.125000,26.052667,33.465000,21.125000,80.437500,0.840252,0
153,120001,2016,10,AC,Acrelândia,NaN,93.87,55.29,41.61,97.44,94.56,23.82,0.0,2.835484,8.988710,26.809677,33.322581,22.991935,83.290323,0.732939,0


### Avaliando os missing values gerados a partir dos joins

In [12]:
dados_final4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27276 entries, 0 to 27275
Data columns (total 21 columns):
ibge                         27276 non-null int64
n_ano                        27276 non-null int64
n_mes                        27276 non-null int64
uf                           27276 non-null object
cidade                       27276 non-null object
total_confirmado_abs         17728 non-null float64
t_agua                       27276 non-null float64
t_banagua                    27276 non-null float64
t_dens                       27276 non-null float64
t_lixo                       27276 non-null float64
t_luz                        27276 non-null float64
agua_esgoto                  27276 non-null float64
evaporacao_piche             27127 non-null float64
insolacao                    27127 non-null float64
precipitacao                 27127 non-null float64
temp_comp_media              27127 non-null float64
temp_maxima                  27127 non-null float64
temp_minima    

Resultado da análise da informação acima:

* Aumento do número de registros da base final:

  Após a junção dos dados da base original com os dados da base auxiliar **`df_ibge_ano_mes`**, a quantidade total de registros subiu de 17728 para 27276 registros.

  Isso era esperado, uma vez que a intenção era gerar novos registros para os meses em que determinado município não possuísse dados.


* Missing Values:

  Os campos relacionados ao município são os únicos sem missing values.

  Após a junção dos dados da base original com os dados das demais bases, restaram registros com missing values nas seguintes colunas:
  * total_confirmado_abs
  * evaporacao_piche
  * insolacao
  * precipitacao
  * temp_comp_media
  * temp_maxima
  * temp_minima
  * umidade_relativa_media
  * velocidade_do_vento_media

  Isso era esperado, pois no caso dos campos de Pluviometria (todos, exceto o campo **`total_confirmado_abs`**), já era sabido que não há dados para o estado de Roraime ("RO") e para alguns meses em outros estados.
  
  No caso da variável resposta (**`total_confirmado_abs`**), os missing values se referem ao aumento de número de registros citado anteriormente.
   
  Em todos esses casos, os campos missing serão tratados na fase de modelagem, permitindo que em cada abordagem se adote um tratamento distinto para esses campos.

### Criando novas colunas com dados dos meses anteriores (lagging)

#### Função auxiliar para o lagging:

In [13]:
def create_lag_columns(df, n_in=1):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        df: Input Pandas DataFrame.
        n_in: Number of lag observations as input (X).
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = df.shape[1]
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s_ant%d' % (df.columns.to_list()[j], i)) for j in range(n_vars)]
    # insert itself at then end of the lags
    cols.append(df)
    names += df.columns.to_list()
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    return agg

#### Criação das colunas de lagging:

Serão criadas novas colunas com a seguinte nomenclatura:

**[nome_coluna]\_ant[numero_meses]**, onde:

  - **[nome_coluna]:** nome da coluna original submetida a lagging;

  - **\_ant:** sufixo indicando que o valor se refere a um mes anterior ao atual;

  - **[numero_meses]:** numero de meses do lagging, ou seja, a quantos meses antes do mes atual o valor se refere;

Exemplo:

    nome_coluna       | descricao
    ----------------- | -------------------------------------------------------------------------------------
    temp_maxima       | coluna com o valor de temp_maxima do mes indicado pelo campo n_mes;
    temp_maxima_ant1  | coluna com o valor de temp_maxima de 1 mes antes do mes indicado pelo campo n_mes;
    temp_maxima_ant5  | coluna com o valor de temp_maxima de 5 meses antes do mes indicado pelo campo n_mes;
    temp_maxima_ant11 | coluna com o valor de temp_maxima de 11 meses antes do mes indicado pelo campo n_mes;

Será criado um dataframe com 11 colunas de lagging (**[nome_coluna]_ant1** .. **[nome_coluna]_ant11**) para cada coluna da lista **`lag_cols`**, incluindo as colunas originais

In [14]:
# lista de colunas que serao submetidas a lagging (todas as que possuem variacao ao longo dos meses)
lag_cols = ['evaporacao_piche', 'insolacao', 'precipitacao', 'temp_comp_media', 'temp_maxima', 'temp_minima',
            'umidade_relativa_media', 'velocidade_do_vento_media', 'google_trends_score', 'total_confirmado_abs']

# tamanho do lag (em meses)
lag_size = 11
# lista de dataframes com o ajuste de lagigng
# isso se faz necessario pra evitar que o lagging de dados de um municipio seja atribuido equivocadamente
# a outro municipio
lst_df_ibge_aux = list()
# para cada cidade, gera as colunas de lagging filtrando o dataset com 
for i in range(len(ibge_list)):
    lst_cols_aux = list()
    for j in range(len(lag_cols)):
        lst_cols_aux.append(create_lag_columns(dados_final4.loc[dados_final4.ibge == ibge_list[i], [lag_cols[j]]], 11))
    lst_df_ibge_aux.append(pd.concat(lst_cols_aux, axis=1))
df_lag_cols = pd.concat(lst_df_ibge_aux, axis=0)
# exibe o resultado parcial
print('dimensao (linhas, colunas):')
print('df_lag_cols:', df_lag_cols.shape)
df_lag_cols

dimensao (linhas, colunas):
df_lag_cols: (27276, 120)


,evaporacao_piche_ant11,evaporacao_piche_ant10,evaporacao_piche_ant9,evaporacao_piche_ant8,evaporacao_piche_ant7,evaporacao_piche_ant6,evaporacao_piche_ant5,evaporacao_piche_ant4,evaporacao_piche_ant3,evaporacao_piche_ant2,evaporacao_piche_ant1,evaporacao_piche,insolacao_ant11,insolacao_ant10,insolacao_ant9,insolacao_ant8,insolacao_ant7,insolacao_ant6,insolacao_ant5,insolacao_ant4,insolacao_ant3,insolacao_ant2,insolacao_ant1,insolacao,precipitacao_ant11,precipitacao_ant10,precipitacao_ant9,precipitacao_ant8,precipitacao_ant7,precipitacao_ant6,precipitacao_ant5,precipitacao_ant4,precipitacao_ant3,precipitacao_ant2,precipitacao_ant1,precipitacao,temp_comp_media_ant11,temp_comp_media_ant10,temp_comp_media_ant9,temp_comp_media_ant8,temp_comp_media_ant7,temp_comp_media_ant6,temp_comp_media_ant5,temp_comp_media_ant4,temp_comp_media_ant3,temp_comp_media_ant2,temp_comp_media_ant1,temp_comp_media,temp_maxima_ant11,temp_maxima_ant10,temp_maxima_ant9,temp_maxima_ant8,temp_maxima_ant7,temp_maxima_ant6,temp_maxima_ant5,temp_maxima_ant4,temp_maxima_ant3,temp_maxima_ant2,temp_maxima_ant1,temp_maxima,temp_minima_ant11,temp_minima_ant10,temp_minima_ant9,temp_minima_ant8,temp_minima_ant7,temp_minima_ant6,temp_minima_ant5,temp_minima_ant4,temp_minima_ant3,temp_minima_ant2,temp_minima_ant1,temp_minima,umidade_relativa_media_ant11,umidade_relativa_media_ant10,umidade_relativa_media_ant9,umidade_relativa_media_ant8,umidade_relativa_media_ant7,umidade_relativa_media_ant6,umidade_relativa_media_ant5,umidade_relativa_media_ant4,umidade_relativa_media_ant3,umidade_relativa_media_ant2,umidade_relativa_media_ant1,umidade_relativa_media,velocidade_do_vento_media_ant11,velocidade_do_vento_media_ant10,velocidade_do_vento_media_ant9,velocidade_do_vento_media_ant8,velocidade_do_vento_media_ant7,velocidade_do_vento_media_ant6,velocidade_do_vento_media_ant5,velocidade_do_vento_media_ant4,velocidade_do_vento_media_ant3,velocidade_do_vento_media_ant2,velocidade_do_vento_media_ant1,velocidade_do_vento_media,google_trends_score_ant11,google_trends_score_ant10,google_trends_score_ant9,google_trends_score_ant8,google_trends_score_ant7,google_trends_score_ant6,google_trends_score_ant5,google_trends_score_ant4,google_trends_score_ant3,google_trends_score_ant2,google_trends_score_ant1,google_trends_score,total_confirmado_abs_ant11,total_confirmado_abs_ant10,total_confirmado_abs_ant9,total_confirmado_abs_ant8,total_confirmado_abs_ant7,total_confirmado_abs_ant6,total_confirmado_abs_ant5,total_confirmado_abs_ant4,total_confirmado_abs_ant3,total_confirmado_abs_ant2,total_confirmado_abs_ant1,total_confirmado_abs
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0

In [15]:
# lista de colunas que NAO serao submetidas a lagging
not_lag_cols = [item for item in dados_final4.columns.to_list() if item not in lag_cols]
# juncao lateral das colunas de lagging ao dataset final
dados_final5 = pd.concat([dados_final4[not_lag_cols], df_lag_cols], axis=1)
# exibe o resultado parcial
print('dimensao (linhas, colunas):')
print('dados_final4[not_lag_cols]:', dados_final4[not_lag_cols].shape)
print('df_lag_cols:', df_lag_cols.shape)
print('dados_final5:', dados_final5.shape)
dados_final5

dimensao (linhas, colunas):
dados_final4[not_lag_cols]: (27276, 11)
df_lag_cols: (27276, 120)
dados_final5: (27276, 131)


,ibge,n_ano,n_mes,uf,cidade,t_agua,t_banagua,t_dens,t_lixo,t_luz,agua_esgoto,evaporacao_piche_ant11,evaporacao_piche_ant10,evaporacao_piche_ant9,evaporacao_piche_ant8,evaporacao_piche_ant7,evaporacao_piche_ant6,evaporacao_piche_ant5,evaporacao_piche_ant4,evaporacao_piche_ant3,evaporacao_piche_ant2,evaporacao_piche_ant1,evaporacao_piche,insolacao_ant11,insolacao_ant10,insolacao_ant9,insolacao_ant8,insolacao_ant7,insolacao_ant6,insolacao_ant5,insolacao_ant4,insolacao_ant3,insolacao_ant2,insolacao_ant1,insolacao,precipitacao_ant11,precipitacao_ant10,precipitacao_ant9,precipitacao_ant8,precipitacao_ant7,precipitacao_ant6,precipitacao_ant5,precipitacao_ant4,precipitacao_ant3,precipitacao_ant2,precipitacao_ant1,precipitacao,temp_comp_media_ant11,temp_comp_media_ant10,temp_comp_media_ant9,temp_comp_media_ant8,temp_comp_media_ant7,temp_comp_media_ant6,temp_comp_media_ant5,temp_comp_media_ant4,temp_comp_media_ant3,temp_comp_media_ant2,temp_comp_media_ant1,temp_comp_media,temp_maxima_ant11,temp_maxima_ant10,temp_maxima_ant9,temp_maxima_ant8,temp_maxima_ant7,temp_maxima_ant6,temp_maxima_ant5,temp_maxima_ant4,temp_maxima_ant3,temp_maxima_ant2,temp_maxima_ant1,temp_maxima,temp_minima_ant11,temp_minima_ant10,temp_minima_ant9,temp_minima_ant8,temp_minima_ant7,temp_minima_ant6,temp_minima_ant5,temp_minima_ant4,temp_minima_ant3,temp_minima_ant2,temp_minima_ant1,temp_minima,umidade_relativa_media_ant11,umidade_relativa_media_ant10,umidade_relativa_media_ant9,umidade_relativa_media_ant8,umidade_relativa_media_ant7,umidade_relativa_media_ant6,umidade_relativa_media_ant5,umidade_relativa_media_ant4,umidade_relativa_media_ant3,umidade_relativa_media_ant2,umidade_relativa_media_ant1,umidade_relativa_media,velocidade_do_vento_media_ant11,velocidade_do_vento_media_ant10,velocidade_do_vento_media_ant9,velocidade_do_vento_media_ant8,velocidade_do_vento_media_ant7,velocidade_do_vento_media_ant6,velocidade_do_vento_media_ant5,velocidade_do_vento_media_ant4,velocidade_do_vento_media_ant3,velocidade_do_vento_media_ant2,velocidade_do_vento_media_ant1,velocidade_do_vento_media,google_trends_score_ant11,google_trends_score_ant10,google_trends_score_ant9,google_trends_score_ant8,google_trends_score_ant7,google_trends_score_ant6,google_trends_score_ant5,google_trends_score_ant4,google_trends_score_ant3,google_trends_score_ant2,google_trends_score_ant1,google_trends_score,total_confirmado_abs_ant11,total_confirmado_abs_ant10,total_confirmado_abs_ant9,total_confirmado_abs_ant8,total_confirmado_abs_ant7,total_confirmado_abs_ant6,total_confirmado_abs_ant5,total_confirmado_abs_ant4,total_confirmado_abs_ant3,total_confirmado_abs_ant2,total_confirmado_abs_ant1,total_confirmado_abs
0,110001,2016,1,RO,Alta Floresta D'Oeste,93.69,80.20,22.58,94.05,93.98,9.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110001,2016,2,RO,Alta Floresta D'Oeste,93.69,80.20,22.58,94.05,93.98,9.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110001,2016,3,RO,Alta Floresta D'Oeste,93.69,80.20,22.58,94.05,93.98,9.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

#### Validação das colunas de lagging:

Para validar o lagging, vamos usar a coluna **`google_trends_score`**, que possui dados agregados por mês (campo **`month`**) e por estado (campo **`sigla_uf`**).

Abaixo, uma amostragem da tabela original utilizada para compor a tabela final:

In [16]:
dados_trends.sort_values(by = ['sigla_uf', 'year', 'month']).head()

,google_trends_score,sigla_uf,month,year
19,44,AC,1,2016
15,53,AC,2,2016
18,93,AC,3,2016
12,44,AC,4,2016
22,38,AC,5,2016


Filtraremos na base final um ano (2016) e um estado (Roraima - RO) e vamos verificar se a soma da coluna **`google_trends_score_ant11`** (que possui o valor da coluna original **`google_trends_score`** de 11 meses antes do mês corrente) equivale ao valor do campo **`google_trends_score`** do mes de janeiro (do ano e estado pesquisado) multiplicado pelo número de municípios deste estado.


Justificativa: Uma vez que a coluna **`google_trends_score_ant11`** terá valor diferente de NaN apenas para o mês de dezembro de cada município e que a sequência de valores dessa coluna ao longo dos meses se repetirá para cada município do mesmo estado, ela é uma boa candidata para validarmos o lagging.

In [17]:
df_teste = dados_final5.loc[(dados_final5.n_ano == 2016) & (dados_final5.uf.str.contains('RO'))]

In [18]:
print('Quantidade de municípios no estado de', df_teste.uf.unique()[0],'=', df_teste.ibge.nunique())

Quantidade de municípios no estado de RO = 12


In [19]:
print('Valor do campo [google_trends_score] no estado de', df_teste.uf.unique()[0],
      'para o mes de janeiro =', df_teste.loc[df_teste.n_mes == 1, 'google_trends_score'].unique()[0])

Valor do campo [google_trends_score] no estado de RO para o mes de janeiro = 35


In [20]:
df_teste[['ibge', 'n_ano', 'n_mes', 'uf', 'cidade', 'google_trends_score_ant11', 'google_trends_score']]

,ibge,n_ano,n_mes,uf,cidade,google_trends_score_ant11,google_trends_score
0,110001,2016,1,RO,Alta Floresta D'Oeste,NaN,35
1,110001,2016,2,RO,Alta Floresta D'Oeste,NaN,85
2,110001,2016,3,RO,Alta Floresta D'Oeste,NaN,88
3,110001,2016,4,RO,Alta Floresta D'Oeste,NaN,40
4,110001,2016,5,RO,Alta Floresta D'Oeste,NaN,46
5,110001,2016,6,RO,Alta Floresta D'Oeste,NaN,16
6,110001,2016,7,RO,Alta Floresta D'Oeste,NaN,23
7,110001,2016,8,RO,Alta Floresta D'Oeste,NaN,40
8,110001,2016,9,RO,Alta Floresta D'Oeste,NaN,93
9,110001,2016,10,RO,Alta Floresta D'Oeste,NaN,71


Conforme dito anteriormente (e verificado acima), o campo **`google_trends_score_ant11`** terá valor diferente de NaN apenas para o mês de dezembro e seu valor será o correspondente ao valor do campo **`google_trends_score`** no mês de janeiro. Esse padrão se repetirá para cada município do mesmo estado.

No caso do estado de Roraima, temos 12 municípios e o valor do campo **`google_trends_score`** no mês de janeiro é 35.

Logo, a soma do campo **`google_trends_score_ant11`** em todo o estado de Roraima deve ser o equivalente a:

12 * 35 = 420

In [21]:
df_teste.google_trends_score_ant11.sum()

420.0

Tudo OK agora!

Podemos salvar o novo dataset com os novos atributos (features) !

### Salvando os dados

In [26]:
# csv
dados_final5.to_csv('../../data/FINAL/lista_dados_sage_FINAL.csv', index=False, sep=';')

### Validando a carga dos dados salvos

In [27]:
# csv
df1 = pd.read_csv('../../data/FINAL/lista_dados_sage_FINAL.csv', sep=';')

In [28]:
df1

,ibge,n_ano,n_mes,uf,cidade,t_agua,t_banagua,t_dens,t_lixo,t_luz,agua_esgoto,evaporacao_piche_ant11,evaporacao_piche_ant10,evaporacao_piche_ant9,evaporacao_piche_ant8,evaporacao_piche_ant7,evaporacao_piche_ant6,evaporacao_piche_ant5,evaporacao_piche_ant4,evaporacao_piche_ant3,evaporacao_piche_ant2,evaporacao_piche_ant1,evaporacao_piche,insolacao_ant11,insolacao_ant10,insolacao_ant9,insolacao_ant8,insolacao_ant7,insolacao_ant6,insolacao_ant5,insolacao_ant4,insolacao_ant3,insolacao_ant2,insolacao_ant1,insolacao,precipitacao_ant11,precipitacao_ant10,precipitacao_ant9,precipitacao_ant8,precipitacao_ant7,precipitacao_ant6,precipitacao_ant5,precipitacao_ant4,precipitacao_ant3,precipitacao_ant2,precipitacao_ant1,precipitacao,temp_comp_media_ant11,temp_comp_media_ant10,temp_comp_media_ant9,temp_comp_media_ant8,temp_comp_media_ant7,temp_comp_media_ant6,temp_comp_media_ant5,temp_comp_media_ant4,temp_comp_media_ant3,temp_comp_media_ant2,temp_comp_media_ant1,temp_comp_media,temp_maxima_ant11,temp_maxima_ant10,temp_maxima_ant9,temp_maxima_ant8,temp_maxima_ant7,temp_maxima_ant6,temp_maxima_ant5,temp_maxima_ant4,temp_maxima_ant3,temp_maxima_ant2,temp_maxima_ant1,temp_maxima,temp_minima_ant11,temp_minima_ant10,temp_minima_ant9,temp_minima_ant8,temp_minima_ant7,temp_minima_ant6,temp_minima_ant5,temp_minima_ant4,temp_minima_ant3,temp_minima_ant2,temp_minima_ant1,temp_minima,umidade_relativa_media_ant11,umidade_relativa_media_ant10,umidade_relativa_media_ant9,umidade_relativa_media_ant8,umidade_relativa_media_ant7,umidade_relativa_media_ant6,umidade_relativa_media_ant5,umidade_relativa_media_ant4,umidade_relativa_media_ant3,umidade_relativa_media_ant2,umidade_relativa_media_ant1,umidade_relativa_media,velocidade_do_vento_media_ant11,velocidade_do_vento_media_ant10,velocidade_do_vento_media_ant9,velocidade_do_vento_media_ant8,velocidade_do_vento_media_ant7,velocidade_do_vento_media_ant6,velocidade_do_vento_media_ant5,velocidade_do_vento_media_ant4,velocidade_do_vento_media_ant3,velocidade_do_vento_media_ant2,velocidade_do_vento_media_ant1,velocidade_do_vento_media,google_trends_score_ant11,google_trends_score_ant10,google_trends_score_ant9,google_trends_score_ant8,google_trends_score_ant7,google_trends_score_ant6,google_trends_score_ant5,google_trends_score_ant4,google_trends_score_ant3,google_trends_score_ant2,google_trends_score_ant1,google_trends_score,total_confirmado_abs_ant11,total_confirmado_abs_ant10,total_confirmado_abs_ant9,total_confirmado_abs_ant8,total_confirmado_abs_ant7,total_confirmado_abs_ant6,total_confirmado_abs_ant5,total_confirmado_abs_ant4,total_confirmado_abs_ant3,total_confirmado_abs_ant2,total_confirmado_abs_ant1,total_confirmado_abs
0,110001,2016,1,RO,Alta Floresta D'Oeste,93.69,80.20,22.58,94.05,93.98,9.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110001,2016,2,RO,Alta Floresta D'Oeste,93.69,80.20,22.58,94.05,93.98,9.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110001,2016,3,RO,Alta Floresta D'Oeste,93.69,80.20,22.58,94.05,93.98,9.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [29]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27276 entries, 0 to 27275
Columns: 131 entries, ibge to total_confirmado_abs
dtypes: float64(125), int64(4), object(2)
memory usage: 27.3+ MB
